Going to start a ground  u p rework fothe data based on features suggested by our favourite co-pilot. It seems like we can get a decent answer based on a lightweight model, by processing the audio files using librosa. The features we're lookoing to extract are: Mel-Frequency Cepstral Coefficients(MFCC) 
Specra Centroid
Specral Bandwidth
Specral Flatness
Spectral Contrast
Time Domain features - Zero Crossing rate (ZCR)
Root Mean Square Energy (RMSE)
Temporal features - Short-time energy
Tempo

Tips for Efficient Processing on Raspberry Pi 5

    Downsampling:
        Use a lower sampling rate (e.g., 16 kHz) unless high-frequency information is crucial.
    Windowing:
        Apply short-time Fourier transform (STFT) with small windows (e.g., 20–50 ms) for manageable computational loads.
    Batch Processing:
        Process audio in chunks to avoid memory and CPU spikes.
    Feature Dimensionality Reduction:
        Reduce the number of features using techniques like Principal Component Analysis (PCA) after extraction.
    Quantized Models:
        Use a lightweight, quantized ML model optimized for edge devices.

Import basic pacages for data processing . We're going to use Librosa to process the data as it seems to be  more lightweight than alternatives.   

In [1]:
import librosa
import numpy as np
from numpy import nan
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import math

Preprocessing of audio, we are resampling, normalising and trimming silence.

Resampling will normalise the sampling rate, reducing one variable between datasets. I assume this means that we will want to resamle audio in the input stream.

Normalise standardises the audio amplitude, apparently mimising the influence of recording volume, to prevent outlier high or low sounds from affecting the  model training. 

Trimming silence to remove useless pieces of data to save on computation, and reduce the affect silence could have on affecting our model weights. (Need to check this, feel like it should be heavily caveated.) I have now checked this, it makes sense that silence being a recurring feature across classes could lead to issues, but having a separate class specifically for silence callsed (no event) would be good potentially.

Going to start with a single wav file as a test.

In [ ]:
TestAudioPath = "audio/fold1/7061-6-0-0.wav"

def extract_features(file_path, sr=16000, n_mfcc=13):
    # Load the audio file
    audio, sample_rate = librosa.load(TestAudioPath, sr=sr)

    # Extract the MFCC features
    mfccs = librosa.features.mfcc(y=aduio, sr=sample_rate, n_mfcc = n_mfcc)

    # Compute the mean of the extracted features across  frames
    mfccs_mean = np.mean(mfccs, axis=1)

    #TODO: This still needs to be finished

I've questioned chatgpt on why it wants  to normalise the MFCCs to a human frequency (using the mel scale.) It has agreed this doesnt really make sense. I think I will try to go down the path of doing both mel scale and none mel scale feature sets, build a model from both and see how they compare. Potentially may be worth stacking them? I don't see how applying a log scale is going to help much though...

In [ ]:
def extract_spectral_features(audio, sr):
    # Short-Time Fourier Transform (STFT)
    stft = np.abs(librosa.stft(audio))
    
    # Spectral Features
    centroid = np.mean(librosa.feature.spectral_centroid(S=stft, sr=sr))
    bandwidth = np.mean(librosa.feature.spectral_bandwidth(S=stft, sr=sr))
    flatness = np.mean(librosa.feature.spectral_flatness(S=stft))
    
    # Temporal Features
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
    peak_amplitude = np.max(np.abs(audio))
    
    # Combine features
    features = [centroid, bandwidth, flatness, peak_amplitude]
    return features